In [1]:
import json, os, time

import pyautogui
from PIL import Image
import cv2
import numpy as np
from dotenv import load_dotenv
import openai

from prompts import *
from GUIUtils import *
from GUIToolbox import LightroomGUIToolBox
from Chat import AgentClient

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

## First Step: Set the window, set clibration points for automated GUI operations
Please Follow the Instruction, if you have done this, please save it for future use.

In [2]:
GUI_config_file = "./GUI_config.json"

if os.path.exists(GUI_config_file):
    print(f"Configuration file '{GUI_config_file}' found. Loading...")
    with open(GUI_config_file, "r") as config_file:
        GUI_config = json.load(config_file)
else:
    # Set the button image for locating button on the screen
    button_image = "./button.png"
    HDPI = 2
    background_color = [26, 26, 26]
    # record the area of the pannel and histogram
    pannel_and_histogram_area = set_calibration_points()
    # record the location of each adjustable element in the pannel area
    slider_positions = record_slider_positions(button_image)
    GUI_config = create_toolbox_config_template(button_image, HDPI, background_color, pannel_and_histogram_area, slider_positions)
    # Save to a configuration file
    with open(GUI_config_file, "w") as config_file:
        json.dump(GUI_config, config_file, indent=4)

Configuration file './GUI_config.json' found. Loading...


### Check the config

### Run test to control the GUI Lightroom

In [3]:
set_slider_positions(test_parameters_2, GUI_config)
print(get_current_parameters(GUI_config))
set_slider_positions(test_parameters_3, GUI_config)
print(get_current_parameters(GUI_config))
set_slider_positions(test_parameters_1, GUI_config)

{'Exposure': 2, 'Contrast': 69, 'Highlights': 69, 'Shadows': 69, 'Whites': 69, 'Blacks': 69, 'Temp': -70, 'Tint': -70, 'Vibrance': -70, 'Saturation': -70}
{'Exposure': -3, 'Contrast': -70, 'Highlights': -70, 'Shadows': -70, 'Whites': -70, 'Blacks': -70, 'Temp': 69, 'Tint': 69, 'Vibrance': 69, 'Saturation': 69}


## Second Init all classes

In [4]:
exp_index = 2
output_dir_path = f"./test-output/test_image{exp_index}"
GUIToolbox = LightroomGUIToolBox(GUI_config, output_dir_path, image_name=f"test_image{exp_index}", clip_history_messages=False)
chat_client = AgentClient(api_key=api_key, toolbox_instance=GUIToolbox, debug=False)
global_style = None

In [5]:
# 分析图像的内容是什么
_ = chat_client.agent_interaction(
    system_prompt=image_content_analyzer_prompt()
)
# 分析图像的直方图现状
histo_style = chat_client.agent_interaction(
    system_prompt=image_global_histo_style_analyzer_prompt(),
    histo_image=True
)
# 根据图片分析，提出一个总体的色彩校正目标
global_style = chat_client.agent_interaction(
    system_prompt=image_global_style_analyzer_prompt(histo_instruction=histo_style)
)
# 根据总体色彩校正目标和可用的图像处理手段，提出初步的校正概念
_ = chat_client.agent_interaction(
    system_prompt=global_retouching_concept_prompt_GUI(global_instruction=global_style)
)
# 并得到矫正计划
_ = chat_client.agent_execute_plan(
    system_prompt=retouching_execute_system_prompt(),
    user_prompt=retouching_execute_user_prompt_GUI(),
    current_function_name='func_to_get_lightroom_adjustment'
)

`func_return_responses`, response: 

**Image Content Analysis:**
This urban photograph prominently features tall skyscrapers in a bustling city intersection. The structure on the right has a distinct glass facade with reflective panels. A visible red logo provides a focal point. Pedestrians and vehicles, including a yellow taxi, populate the scene, contributing to the vitality of the area.

**Visual and Emotional Analysis:**  
The angle provides an engaging, upward view that highlights towering architecture, signifying urban density and vibrancy. Lighting lends a natural tone, though shadows dominate parts of the image, creating a lively yet slightly subdued atmosphere. The interplay of glass reflections and red logo evokes modernity and corporate influence, while the bustling street levels ground the shot in everyday life.
`func_return_responses`, response: 

### Histogram Analysis:
The histogram shows peaks in the shadows (left) and highlights (right), with the midtones gradually dec

In [6]:
while True:
    _ = chat_client.agent_reflection_plan(
        system_prompt=retouching_reflection_system_prompt(),
        combine_image=False
    )
    
    if GUIToolbox.satisfactory_status:
        break

    _ = chat_client.agent_execute_plan(
    system_prompt=retouching_execute_system_prompt(),
    user_prompt=retouching_execute_user_prompt_GUI(),
    current_function_name='func_to_get_lightroom_adjustment'
    )

The adjustment with parameter {'Exposure': 0.2, 'Contrast': 15, 'Highlights': -30, 'Shadows': 25, 'Whites': 10, 'Blacks': -10, 'Temp': 10, 'Tint': 0, 'Vibrance': 20, 'Saturation': 10} is NOT satisfactory, reason: The prominent glare on the glass structure and overly contrasting shadows detract from a balanced visual presentation. While certain colors like red logos stand out, the overall shadow and midtone improvements need further refinement for vibrancy without overemphasis..
`func_to_get_lightroom_adjustment`, response: 

{'Exposure': 0.3, 'Contrast': 20, 'Highlights': -40, 'Shadows': 30, 'Whites': 15, 'Blacks': -15, 'Temp': 15, 'Tint': 0, 'Vibrance': 25, 'Saturation': 10} and the reason is: These adjustments aim to emphasize the architectural details and highlight urban vibrancy. The slight exposure lift ensures better visibility, while increasing contrast and balancing highlights and shadows fine-tunes structure and depth. The warmth, vibrance, and saturation enhance color presenc